# Import data and train model

In [ ]:
import tensorflow_datasets as tfds
import tensorflow as tf
import numpy as np

fashion_mnist = tf.keras.datasets.fashion_mnist.load_data()
(X_train_full, Y_train_full), (X_test, Y_test) = fashion_mnist
X_train, Y_train = X_train_full[:-5000], Y_train_full[:-5000]
X_valid, Y_valid = X_train_full[-5000:], Y_train_full[-5000:]

In [ ]:
print("Shape of Training Set {} {}".format(X_train.shape, Y_train.shape))
print("Shape of Validation Set {} {}".format(X_valid.shape, Y_valid.shape))
print("Shape of Test Set {} {}".format(X_test.shape, Y_test.shape))

In [ ]:
#For simplicity scalling the intensity of piexels to 0-1 range
X_train, X_valid, X_test = X_train/255., X_valid/255., X_test/255

In [ ]:
#Get class names
class_names = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat", "Sandal", "Shirt", "Sneaker", "Bag", "Ankle Boot"]

#print lablel of first image from train set
print(class_names[Y_train[0]])

In [ ]:
print(X_train.shape[1:])

In [ ]:
#Creating model using sequential API
tf.random.set_seed(42)

model1 = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=X_train.shape[1:]), #no need to provide input layer
    tf.keras.layers.Dense(300, activation="relu"),
    tf.keras.layers.Dense(100, activation="relu"),
    tf.keras.layers.Dense(10, activation="softmax"),
])


In [ ]:
#Building complex model using Fuctional API
tf.random.set_seed(42)

flatten_layer = tf.keras.layers.Flatten()
normalization_layer = tf.keras.layers.Normalization()
hidden_layer1 = tf.keras.layers.Dense(300, activation="relu")
hidden_layer2 = tf.keras.layers.Dense(100, activation="relu")
concat_layer = tf.keras.layers.Concatenate()
output_layer = tf.keras.layers.Dense(10, activation="softmax")

input_ = tf.keras.layers.Input(shape=X_train.shape[1:])
flatten = flatten_layer(input_)
normalized = normalization_layer(flatten)
hidden1 = hidden_layer1(normalized)
hidden2 = hidden_layer2(hidden1)
concat = concat_layer([normalized,hidden2])
output = output_layer(concat)

model2 = tf.keras.Model(inputs=[input_], outputs=[output])

In [ ]:
#Building Model using Subclassing API
class CustomModel(tf.keras.Model):
    def __init__(self, units1=300, units2=100, units3=10, activation_hidden="relu", activation_output="softmax", **kwargs):
        super().__init__(**kwargs) #needed to support naming the model
        self.flatten_layer = tf.keras.layers.Flatten()
        self.normalization_layer = tf.keras.layers.Normalization()
        self.hidden_layer1 = tf.keras.layers.Dense(units1, activation=activation_hidden)
        self.hidden_layer2 = tf.keras.layers.Dense(units2, activation=activation_hidden)
        self.concat_layer = tf.keras.layers.Concatenate()
        self.output_layer = tf.keras.layers.Dense(units3, activation=activation_output)
        
    def call(self,inputs):
        flatten = self.flatten_layer(inputs)
        normalized = self.normalization_layer(flatten)
        hidden1 = self.hidden_layer1(normalized)
        hidden2 = self.hidden_layer2(hidden1)
        concat = self.concat_layer([normalized,hidden2])
        output = self.output_layer(concat)
        return output
    
model3 = CustomModel(300,100,10,"relu","softmax",name="my_custom_model")

**Select The Model in Below Cell**

In [ ]:
#refine model
model = model1

In [ ]:
# Model Layer Weights or Biases can only be printed for Sequencial or Functional API
'''
#Checking modlel layers and model weight and biases
print("Model Layers are")
print(model.layers)

print("\n Weights of First Layer are")
print(model.layers[-1].get_weights()[0])

print("\n Biases of First Layer are")
print(model.layers[-1].get_weights()[1])
'''

In [ ]:
#Compiling and Fit Model
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

checkpoint_cb = tf.keras.callbacks.ModelCheckpoint("my_checkpoints", save_weights_only=True)
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)

model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
history = model.fit(X_train, Y_train, epochs=30, validation_data=(X_valid, Y_valid), callbacks=[checkpoint_cb, early_stopping_cb])

In [ ]:
#Plot training and Validation loss
import matplotlib.pyplot as plt
import pandas as pd

pd.DataFrame(history.history).plot(
    figsize=(8,5), grid=True, xlabel="Epoch", style=["r--","r--.","b-","b-."]
)

In [ ]:
#Evaluate model
loss, accuracy = model.evaluate(X_test,Y_test)
print("Test Accuracy is %.2f" %(accuracy))

In [ ]:
#Make Predictions on random test data
for _ in range(10):
    random_number = np.random.randint(low=1, high=1000)
    model_prediction = model.predict(X_test[random_number:random_number+1])
    predicted_category = class_names[model_prediction.reshape(-1).argmax()]

    actual_category = class_names[int(Y_test[random_number:random_number+1])]

    print("Actual Category : {} \nPredicted Category : {}".format(actual_category, predicted_category))
    image = X_test[random_number:random_number+1].reshape(28,28)
    plt.figure(figsize=(0.8,0.8))
    plt.imshow(image, cmap="binary")
    plt.axis('off')
    plt.show()